In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0419 02:22:25.218207922    5145 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0419 02:22:32.113872559    5145 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [3]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [4]:
pbar = tqdm()

0it [00:00, ?it/s]

#### Run all operators

In [5]:
fs_list = fs.ls(f'{utils.GCS_FILE_PATH}rt_trips/')

In [8]:
ran_operators = [int(path.split('rt_trips/')[1].split('_')[0])
                 for path in fs_list
                 if path.split('rt_trips/')[1]]

In [7]:
not_ran_operators = []

for agency in air_joined.calitp_itp_id.unique():
# for agency in [273]:
    if agency in ran_operators:
        print(f'already ran: {agency}')
        continue
    analysis_date = dt.date(2022, 3, 30) ##wednesday, new tables
    ## https://docs.google.com/spreadsheets/d/16tcL3fPdYkrNajDNneSR1b5ImhFOCJ0gWFnEaXyg16A/edit#gid=0
    if agency == 273:
        analysis_date = dt.date(2022, 2, 8) # #override for SacRT 
    day = str(analysis_date.day).zfill(2)
    month = str(analysis_date.month).zfill(2)
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        delay_to_parquet = rt_day.stop_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet['arrival_time'] = delay_to_parquet.arrival_time.map(lambda x: x.isoformat())
        delay_to_parquet['actual_time'] = delay_to_parquet.actual_time.map(lambda x: x.isoformat())
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        shared_utils.utils.geoparquet_gcs_export(delay_to_parquet,
                                         f'{utils.GCS_FILE_PATH}stop_delay_views/',
                                        f'{agency}_{month}_{day}'
                                        )
        # rt_day.set_filter(start_time='15:00', end_time='19:00')
        # m = rt_day.segment_speed_map(how='low_speeds', size = [1300, 700])
        # shared_utils.utils.geoparquet_gcs_export(rt_day.stop_segment_speed_view,
        #                                          f'{utils.GCS_FILE_PATH}segment_speed_views/',
        #                                         f'{agency}_{month}_{day}_pm_peak'
        #                                         )
        rt_day.rt_trips.to_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{agency}_{month}_{day}.parquet') ## early step towards scaling
        # m.save(f'./speedmaps/{agency}_02_{day}_pm_peak.html')
        # rt_day.stop_segment_speed_view.to_parquet(f'{utils.GCS_FILE_PATH}segment_speed_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
        delay_to_parquet = rt_day.endpoint_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        shared_utils.utils.geoparquet_gcs_export(delay_to_parquet,
                                         f'{utils.GCS_FILE_PATH}endpoint_delay_views/',
                                        f'{agency}_{month}_{day}_pm_peak'
                                        )
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 484...
rt failed for agency 484
no vehicle positions data found for 2022-03-30
calculating for agency: 380...
found parquet
found parquet
found parquet
found parquet
rt failed for agency 380
vehicle positions trip ids not in schedule
already ran: 300
calculating for agency: 183...
getting trips...


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km 

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

complete for agency: 183
already ran: 182
already ran: 243
already ran: 295
already ran: 170
already ran: 45
calculating for agency: 260...
getting trips...


...

complete for agency: 312
already ran: 126
already ran: 361
already ran: 188
already ran: 148
calculating for agency: 99...
getting trips...


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
could not generate delays for trip t_74580_b_33474_tn_7
'NoneType' object has no attribute 'split'
could not generate delays for trip t_74580_b_33474_tn_11
'NoneType' object has no attribute 'split'
could not generate delays for trip t_74580_b_33474_tn_5
'NoneType' object has no attribute 'split'
could not generate delays for trip t_74580_b_33474_tn_10
'NoneType' object has no attribute 'split'
could not generate delays for trip t_74580_b_33474_tn_8
'NoneType' object has no attribute 'split'
could not generate delays for trip t_74580_b_33474_tn_3
'NoneType' object has no attribute 'split'
could not generate delays for trip t_74580_b_33474_tn_4
'NoneType' object has no attribute 'split'
could not generate delays for trip t_74580_b_33474_tn_2
'NoneType' object has no attribute 'split'
could not generate delays for trip

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

complete for agency: 99
already ran: 226
already ran: 278
already ran: 259


In [9]:
import yaml

In [10]:
with open('../reports_config.yml') as reports_config:
    reports_config_data = yaml.load(reports_config, yaml.Loader)

In [11]:
reports_config_data

{'dla': {'notebook': './dla/e76_obligated_funds/prototype-district.ipynb',
  'params': {'district': [4, 7]}},
 'rt': {'notebook': './rt_delay/15_parameterize.ipynb',
  'params': {'itp_id': [110,
    126,
    127,
    135,
    148,
    159,
    167,
    170,
    182,
    188,
    194,
    208,
    218,
    221,
    226,
    235,
    243,
    246,
    259,
    269,
    273,
    278,
    282,
    284,
    290,
    293,
    294,
    295,
    300,
    301,
    30,
    310,
    315,
    336,
    349,
    350,
    361,
    45,
    4,
    75]}}}

In [12]:
reports_config_data['rt']['params']['itp_id'] = ran_operators

In [13]:
output = yaml.dump(reports_config_data)

In [14]:
# reports_config_data

In [15]:
with open('../reports_config.yml', 'w') as reports_config:
    reports_config.write(output)

### Ongoing issues
* 358 Union City too many trips without shape?

#### Parameters
* 159 1+ maps don't render
* 167 map_from_metrics fails

In [17]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    # >> filter(_.itp_id == itp_id)
    >> collect()
)

In [26]:
airtable_organizations = airtable_organizations.dropna(subset=['itp_id']) >> mutate(itp_id = _.itp_id.astype('int64'))

In [38]:
airtable_organizations = airtable_organizations >> filter(_.itp_id.isin(ran_operators))

In [39]:
airtable_organizations.head(3)

,itp_id,name,caltrans_district,website,ntp_id,drmt_organization_name
526,194,Marin County Transit District,04 - Oakland,None,['recgeJopomk1yuLGl'],None
542,315,Sonoma-Marin Area Rail Transit District,04 - Oakland,None,['recguX2t9zxBKD7RR'],None
549,293,Santa Barbara Metropolitan Transit District,05 - San Luis Obispo,None,['recJ5AvgWoFFVR0SF'],None


In [31]:
with open('../new_reports_config.yml') as reports_config:
    reports_config_data = yaml.load(reports_config, yaml.Loader)

In [46]:
# reports_config_data = {}

In [42]:
for district in airtable_organizations.caltrans_district.unique():
    filtered = airtable_organizations >> filter(_.caltrans_district == district)
    reports_config_data[f'District {district}'] = {'notebook': './rt_delay/15_parameterize.ipynb',
  'params': {'itp_id': filtered.itp_id.to_list()}}

In [43]:
reports_config_data

{'District 04 - Oakland': {'notebook': './rt_delay/15_parameterize.ipynb',
  'params': {'itp_id': [194,
    315,
    246,
    167,
    110,
    350,
    310,
    336,
    282,
    4,
    127,
    301,
    294,
    218,
    290]}},
 'District 05 - San Luis Obispo': {'notebook': './rt_delay/15_parameterize.ipynb',
  'params': {'itp_id': [293, 208]}},
 'District 02 - Redding': {'notebook': './rt_delay/15_parameterize.ipynb',
  'params': {'itp_id': [259]}},
 'District 07 - Los Angeles': {'notebook': './rt_delay/15_parameterize.ipynb',
  'params': {'itp_id': [75, 182, 45, 243, 300, 170, 295]}},
 'District 06 - Fresno': {'notebook': './rt_delay/15_parameterize.ipynb',
  'params': {'itp_id': [188, 126, 148, 361, 188]}},
 'District 11 - San Diego': {'notebook': './rt_delay/15_parameterize.ipynb',
  'params': {'itp_id': [278, 226, 278]}},
 'District 03 - Marysville': {'notebook': './rt_delay/15_parameterize.ipynb',
  'params': {'itp_id': [221, 273]}},
 'District 10 - Stockton': {'notebook': './

In [44]:
output = yaml.dump(reports_config_data)

In [45]:
with open('../new_reports_config.yml', 'w') as reports_config:
    reports_config.write(output)

In [15]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')